In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(os.getcwd())

D:\Python Codes\Rent Predictor


In [3]:
entries=os.listdir()
print(entries)

['.ipynb_checkpoints', 'HouseRent.ipynb', 'houserentdhaka.csv', 'Testing.ipynb', 'Untitled.ipynb']


In [4]:
df=pd.read_csv(entries[2])

In [5]:
df.head()

,Unnamed: 0,Location,Area,Bed,Bath,Price
0,0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand
3,3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand
4,4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand


In [6]:
df.shape

(28800, 6)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  28800 non-null  int64 
 1   Location    28800 non-null  object
 2   Area        28800 non-null  object
 3   Bed         28800 non-null  int64 
 4   Bath        28800 non-null  int64 
 5   Price       28800 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.3+ MB


In [8]:
df.columns

Index(['Unnamed: 0', 'Location', 'Area', 'Bed', 'Bath', 'Price'], dtype='object')

In [9]:
df.rename({'Price':'Rent'},axis=1, inplace=True)

In [10]:
df.drop(columns=df.columns[0], axis=1,inplace=True)

In [11]:
df.head()

,Location,Area,Bed,Bath,Rent
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand
3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand
4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand


In [12]:
df.dtypes

Location    object
Area        object
Bed          int64
Bath         int64
Rent        object
dtype: object

In [13]:
df['Location'].head()

0                      Block H, Bashundhara R-A, Dhaka
1                             Farmgate, Tejgaon, Dhaka
2    Block B, Nobodoy Housing Society, Mohammadpur,...
3                            Gulshan 1, Gulshan, Dhaka
4                                     Baridhara, Dhaka
Name: Location, dtype: object

In [14]:
df['Location']=df['Location'].str.split(',').str[:-1]

df['Location'] = df['Location'].str[-1]

df['Location'] = df['Location'].str.strip()

df['Location'].head()

0    Bashundhara R-A
1            Tejgaon
2        Mohammadpur
3            Gulshan
4          Baridhara
Name: Location, dtype: object

In [15]:
df['Area']=df['Area'].str.split(' ').str[:-1]

In [16]:
df['Area'].head()

0    [1,600]
1      [900]
2    [1,250]
3    [2,200]
4    [2,200]
Name: Area, dtype: object

In [17]:
df['Area'] = df['Area'].apply(lambda x: ''.join(map(str, x)))

In [18]:
df['Area'] = df['Area'].str.replace(',','')

In [19]:
df['Area'] = df['Area'].astype(int)

In [20]:
df['Area'].head()

0    1600
1     900
2    1250
3    2200
4    2200
Name: Area, dtype: int32

In [21]:
print(df['Area'].dtype)

int32


In [23]:
df['Area'] = df['Area'].astype("int64")

In [24]:
print(df['Area'].dtypes)

int64


In [25]:
df['Area'].head()

0    1600
1     900
2    1250
3    2200
4    2200
Name: Area, dtype: int64

In [27]:
df['Rent']=df['Rent'].replace({"Thousand":"*1e3", "Lakh":"*1e5"}, regex=True).map(pd.eval).astype("int64")

df['Rent'].head()

0    20000
1    20000
2    18000
3    75000
4    75000
Name: Rent, dtype: int64

In [28]:
df.head()

,Location,Area,Bed,Bath,Rent
0,Bashundhara R-A,1600,3,3,20000
1,Tejgaon,900,2,2,20000
2,Mohammadpur,1250,3,3,18000
3,Gulshan,2200,3,4,75000
4,Baridhara,2200,3,3,75000


In [29]:
df.dtypes

Location    object
Area         int64
Bed          int64
Bath         int64
Rent         int64
dtype: object

In [30]:
df.isna().sum()

Location    13
Area         0
Bed          0
Bath         0
Rent         0
dtype: int64

In [31]:
df.dropna(inplace=True)